In [2]:
from deepface import DeepFace

2024-04-27 15:23:46.300595: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-27 15:23:47.131292: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
DeepFace.analyze(img_path="/home/akshilmy/MobileApps/health_companion_app/server/temp/8d272092_akshilmy_face/0113.jpg",actions=['emotion'])

2024-04-27 15:24:01.705658: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-27 15:24:01.749761: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[{'emotion': {'angry': 0.0004766885297069707,
   'disgust': 2.521355431596674e-16,
   'fear': 5.020502713044758e-06,
   'happy': 4.2696596027909145e-08,
   'sad': 0.00741579395799042,
   'surprise': 1.8283464374808676e-09,
   'neutral': 99.99210238409653},
  'dominant_emotion': 'neutral',
  'region': {'x': 176,
   'y': 708,
   'w': 697,
   'h': 697,
   'left_eye': (656, 978),
   'right_eye': (400, 995)},
  'face_confidence': 0.91}]

In [4]:
import json
import random

def load_questions_from_files(file_paths):
    questions = []
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            data = json.load(file)
            questions.extend(data['questions'])
    return questions

import random

def choose_random_questions(questions, total_questions):
    random.shuffle(questions)
    selected_questions = []
    remaining_questions = total_questions
    prev_tag = None  # To keep track of the previous tag
    while remaining_questions > 0:
        random.shuffle(questions)
        for question in questions:
            # Check if the question's tag is different from the previous one
            if prev_tag != question['tag']:
                if remaining_questions == 0:
                    break
                selected_questions.append(question)
                remaining_questions -= 1
                prev_tag = question['tag']
    return selected_questions


# Example usage
file_paths = ['/home/akshilmy/MobileApps/health_companion_app/server/app/questionnaire/image_feedback.json',
              '/home/akshilmy/MobileApps/health_companion_app/server/app/questionnaire/jokes.json',
              '/home/akshilmy/MobileApps/health_companion_app/server/app/questionnaire/mental_health.json'
             ]
total_questions = 10 # Total number of questions you want to select
questions = load_questions_from_files(file_paths)
selected_questions = choose_random_questions(questions, total_questions)

# Print the selected questions
for i, question in enumerate(selected_questions, start=1):
    print(f"{i}. {question['question']}")


1. How often have you felt down, depressed, or hopeless in the past two weeks?
2. Here's a joke. What did the ocean say to the beach?
3. How often have you had trouble falling asleep, staying asleep, or sleeping too much?
4. Here's a joke. Why did the tomato turn red?
5. How does this image make you feel?
6. Here's a joke. Why don't skeletons fight each other?
7. Have you had thoughts of harming yourself or thoughts of suicide?
8. Here's a joke. What do you call fake spaghetti?
9. What do you think the main subject of this image is thinking or feeling?
10. Here's a joke. Why did the golfer bring two pairs of pants?


In [5]:
questions

[{'id': 'health0',
  'question': 'How often have you felt down, depressed, or hopeless in the past two weeks?',
  'tag': 2,
  'options': {'Not at all': True,
   'Several days': False,
   'More than half the days': False,
   'Nearly every day': False}},
 {'id': 'joke7',
  'question': "Here's a joke. What did the ocean say to the beach?",
  'tag': 1,
  'options': {'Nothing, it just waved!': True,
   'Why are you so sandy?': True,
   "I don't find it funny.": False,
   "I can't stop laughing!": True}},
 {'id': 'health3',
  'question': 'How often have you had trouble falling asleep, staying asleep, or sleeping too much?',
  'tag': 2,
  'options': {'Not at all': True,
   'Several days': False,
   'More than half the days': False,
   'Nearly every day': False}},
 {'id': 'joke6',
  'question': "Here's a joke. Why did the tomato turn red?",
  'tag': 1,
  'options': {'Because it saw the salad dressing!': True,
   'Because it was embarrassed.': True,
   "I don't find it funny.": False,
   "I'm r

In [33]:
answers = [
    {"id": "joke7", "response": "Why are you so sandy?"},
    {"id": "health4", "response": "Nearly every day"},
    {"id": "img0", "response": "Sad"},
    {"id": "health1", "response": "Nearly every day"},
    {"id": "joke9", "response": "Stop staring ats me."},
    {"id": "joke4", "response": "I don't find it funny."},
    {"id": "joke6", "response": "I don't find it funny."},
    {"id": "joke11", "response": "I don't find it funny."},
    {"id": "health0", "response": "Nearly every day"},
    {"id": "health6", "response": "Nearly every day"},
]


In [37]:
def evaluate_answers(questions, answers):
    results = []

    for answer in answers:
        question_id = answer['id']
        response = answer['response']
        question_found = False

        for question in questions:
            if question['id'] == question_id:
                question_found = True
                if 'options' in question:
                    options = question['options']
                    if response in options and options[response]:
                        results.append(True)
                    else:
                        results.append(False)

        if not question_found:
            results.append(False)  # If the question ID is not found, consider the answer incorrect

    return results

# Assuming 'questions' and 'answers' are the provided JSON data
results = evaluate_answers(questions, answers)
print(results)
print(results.count(False)/len(results))

[True, False, False, False, True, False, False, False, False, False]
0.8


In [2]:
converted_answers = {
    "joke7": "Why are you so sandy?",
    "health4": "Nearly every day",
    "img0": "Sad",
    "health1": "Nearly every day",
    "joke9": "Stop staring ats me.",
    "joke4": "I don't find it funny.",
    "joke6": "I don't find it funny.",
    "joke11": "I don't find it funny.",
    "health0": "Nearly every day",
    "health6": "Nearly every day",
}

In [6]:
def evaluate_answers(questions, answers):
    results = []

    for question_id in answers:
        response = answers[question_id]
        question_found = False
        for question in questions:
            if question['id'] == question_id:
                question_found = True
                if 'options' in question:
                    options = question['options']
                    if response in options and options[response]:
                        results.append(True)
                    else:
                        results.append(False)

        if not question_found:
            results.append(False)  # If the question ID is not found, consider the answer incorrect

    return results

# Assuming 'questions' and 'answers' are the provided JSON data
results = evaluate_answers(questions, converted_answers)
print(results)
print(results.count(False)/len(results))

[True, False, False, False, False, False, False, False, False, False]
0.9


In [5]:
def get_emotion_percentages(data):
  # Remove neutral emotion from calculation
  emotions = {key: value for key, value in data['emotion'].items() if key != 'neutral'}
  total_emotion_value = sum(emotions.values())

  # Calculate emotion percentages
  emotion_percentages = {key: (value / total_emotion_value) * 100 for key, value in emotions.items()}

  # Find dominant emotion
  dominant_emotion = max(emotion_percentages, key=emotion_percentages.get)
  dominant_emotion_percentage = emotion_percentages[dominant_emotion]

  return {
      'emotion': emotion_percentages,
      'dominant_emotion': dominant_emotion,
      'dominant_emotion_percentage': dominant_emotion_percentage
  }

# Example usage
data = {'emotion': {'angry': 0.32598969992250204, 'disgust': 1.0617613559193995e-09, 'fear': 0.0023397180484607816, 'happy': 0.004592642653733492, 'sad': 1.0442350059747696, 'surprise': 3.950582794232105e-06, 'neutral': 198.6228346824646}, 'dominant_emotion': 'neutral', 'result': 0.0}

result = get_emotion_percentages(data)
print(result)


{'emotion': {'angry': 23.67113907552816, 'disgust': 7.709783691621048e-08, 'fear': 0.16989429830391867, 'happy': 0.33348625127288595, 'sad': 75.825193433535, 'surprise': 0.00028686426219567116}, 'dominant_emotion': 'sad', 'dominant_emotion_percentage': 75.825193433535}


In [10]:
def get_emotion_percentages(data):
  """
  This function takes an emotion dictionary as input and returns a dictionary with the emotion percentages, dominant emotion, second dominant emotion, and their corresponding percentages.

  Args:
      data: A dictionary containing the emotion data.

  Returns:
      A dictionary with the following keys:
          emotion_percentages: A dictionary containing the emotion percentages.
          dominant_emotion: The dominant emotion.
          dominant_emotion_percentage: The dominant emotion percentage.
          second_dominant_emotion: The second dominant emotion (if it exists).
          second_dominant_emotion_percentage: The second dominant emotion percentage (if it exists).
  """
  # Remove neutral emotion from calculation
  emotions = {key: value for key, value in data['emotion'].items()}
  total_emotion_value = sum(emotions.values())

  # Calculate emotion percentages
  emotion_percentages = {key: (value / total_emotion_value) * 100 for key, value in emotions.items()}

  # Sort emotions by percentage in descending order
  sorted_emotions = dict(sorted(emotion_percentages.items(), key=lambda item: item[1], reverse=True))

  # Get dominant and second dominant emotions (if it exists)
  dominant_emotion = next(iter(sorted_emotions))
  dominant_emotion_percentage = sorted_emotions[dominant_emotion]
  second_dominant_emotion = None
  second_dominant_emotion_percentage = None

  # Check if there's a second dominant emotion (excluding the dominant one)
  if len(sorted_emotions) > 1:
    second_dominant_emotion = next(iter(sorted_emotions.keys()))  # Skip the first element (dominant)
    second_dominant_emotion_percentage = list(sorted_emotions.values())[1]

  return {
      'emotion_percentages': emotion_percentages,
      'dominant_emotion': dominant_emotion,
      'dominant_emotion_percentage': dominant_emotion_percentage,
      'second_dominant_emotion': second_dominant_emotion,
      'second_dominant_emotion_percentage': second_dominant_emotion_percentage
  }

# Example usage
data = {'emotion': {'angry': 0.32598969992250204, 'disgust': 1.0617613559193995e-09, 'fear': 0.0023397180484607816, 'happy': 0.004592642653733492, 'sad': 1.0442350059747696, 'surprise': 3.950582794232105e-06, 'neutral': 198.6228346824646}, 'dominant_emotion': 'neutral', 'result': 0.0}

result = get_emotion_percentages(data)
print(result)


{'emotion_percentages': {'angry': 0.16299485346506287, 'disgust': 5.308806893717535e-10, 'fear': 0.0011698590493782156, 'happy': 0.0022963213762295194, 'sad': 0.5221175142110615, 'surprise': 1.9752914395778195e-06, 'neutral': 99.31141947607595}, 'dominant_emotion': 'neutral', 'dominant_emotion_percentage': 99.31141947607595, 'second_dominant_emotion': 'neutral', 'second_dominant_emotion_percentage': 0.5221175142110615}
